## News Summary Agent with ReAct + LangChain

This notebook demonstrates a simple ReAct-style agent built with LangChain that:

- Uses a search tool to find recent articles on a given topic  
- Summarizes each article  
- Estimates reading time and reading level  
- Outputs a markdown-formatted summary for each article

*This project was inspired in part by ideas from the [LangChain course on Udemy](https://www.udemy.com/course/langchain).*

In [21]:
from typing import Union, List
from dotenv import load_dotenv
import textwrap
import json

from langchain.agents import tool
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import Tool
from langchain.tools.render import render_text_description
from langchain_community.tools.tavily_search import TavilySearchResults

from callbacks import AgentCallbackHandler

load_dotenv()

True

## Tool Overview

This section defines a set of reusable tools that the agent can call to process news article content. These tools are wrapped using the `@tool` decorator from LangChain, making them accessible to the agent for specific tasks during the reasoning process:

- **`TavilySearchResults`**  
  A search tool that retrieves up to 3 relevant articles from the web based on a user query. It's used to fetch current news or topic-specific content.

- **`summarize_text`**  
  A simple summarization tool that truncates a block of text to 300 characters using Python’s built-in `textwrap` module. It ensures the agent provides concise responses.

- **`estimate_reading_time`**  
  Estimates how long it would take to read a given passage based on an average reading speed of 200 words per minute. The output is a rounded time in minutes.

- **`estimate_reading_level`**  
  Calculates the approximate U.S. grade level needed to understand a given text using the Flesch-Kincaid readability formula. It heuristically counts syllables and sentence lengths to determine complexity.

These tools are invoked by the agent as it reasons through the steps of answering a user's question about recent news articles.

In [22]:
# -------------------------
# TOOL DEFINITIONS
# -------------------------

search = TavilySearchResults(max_results=3)


@tool
def summarize_text(text: str) -> str:
    """Summarizes a block of text into a few sentences."""
    return textwrap.shorten(text, width=300, placeholder="...")


@tool
def estimate_reading_time(text: str) -> str:
    """Estimates reading time in minutes given a block of text."""
    words = len(text.split())
    minutes = max(1, round(words / 200))  # 200 wpm
    return f"{minutes} minute(s)"


@tool
def estimate_reading_level(text: str) -> str:
    """Rudimentary reading level estimator using sentence and word complexity."""
    sentences = max(1, text.count('.') + text.count('!') + text.count('?'))
    words = len(text.split())
    syllables = sum(len([c for c in word if c.lower() in "aeiouy"]) for word in text.split())
    asl = words / sentences  # average sentence length
    asw = syllables / words  # average syllables per word
    grade = (0.39 * asl) + (11.8 * asw) - 15.59  # Flesch-Kincaid grade
    return f"Grade level: {round(grade, 1)}"

## Utility Functions Overview

This section defines helper functions that support the agent's ability to dynamically locate and invoke tools during the reasoning and execution process.

- **`find_tool_by_name(tools, tool_name)`**  
  Searches through a list of available tools and returns the one that matches the given name. Raises a `ValueError` if no match is found. This is useful for selecting the appropriate tool based on the agent’s current action step.

- **`invoke_tool(tool, tool_input)`**  
  A wrapper function that attempts to call a tool's `.func()` method if available (common for tools wrapped with LangChain’s `@tool` decorator), or falls back to calling the tool object directly. This adds flexibility for invoking different tool types.

- **`if tool_name == "tavily_search_results_json":`**  
  This conditional captures the raw JSON output from the Tavily search tool. The result (`observation`) is saved to `tavily_raw` for use in final formatting after all reasoning steps are complete.

These utilities enable the agent to flexibly and reliably interact with tools while constructing a multi-step answer.

In [23]:
# -------------------------
# UTILITY FUNCTIONS
# -------------------------

def find_tool_by_name(tools: List[Tool], tool_name: str) -> Tool:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    raise ValueError(f"Tool '{tool_name}' not found.")


def invoke_tool(tool: Tool, tool_input: str) -> str:
    try:
        return tool.func(tool_input)
    except AttributeError:
        return tool(tool_input)

if tool_name == "tavily_search_results_json":
    tavily_raw = observation  # capture raw JSON for later formatting

## Main Agent Loop Overview

This section initializes and runs the agent that processes user input to retrieve, analyze, and summarize recent news articles using LangChain's ReAct framework.

### Key Components:

- **Agent Setup:**
  - A prompt template is defined to enforce a strict ReAct format: `Question → Thought → Action → Observation → Final Answer`.
  - The final output is expected to be a markdown-formatted list with hyperlinks, summaries (max 20 words), reading levels, and reading times.
  - Tools (`search`, `summarize_text`, `estimate_reading_time`, `estimate_reading_level`) are passed into the agent.

- **LLM and Prompt Pipeline:**
  - A `ChatOpenAI` model is instantiated and linked with the prompt and output parser (`ReActSingleInputOutputParser`).
  - The prompt is dynamically formatted with tool names and descriptions.
  - The agent is assembled using LangChain’s operator (`|`) syntax.

- **User Interaction:**
  - The user is prompted for a topic of interest.
  - This topic is used to formulate a search-and-summarize question, which is passed to the agent.

- **Agent Execution Loop:**
  - The loop continues while the agent produces `AgentAction` steps (i.e., tool invocations).
  - For each action:
    - The relevant tool is located and invoked.
    - The result is printed and appended to the `intermediate_steps` scratchpad.

- **Final Output Formatting:**
  - Once the agent concludes with a `FinalAnswer`, the system attempts to:
    - Parse the Tavily search results JSON.
    - Format each article into a markdown bullet point with:
      - A clickable title
      - A one-sentence summary (max 20 words)
      - Estimated reading time
      - Estimated reading level

If the results are not from Tavily or parsing fails, a fallback message or raw output is shown.

In [24]:
# -------------------------
# MAIN AGENT LOOP
# -------------------------

if __name__ == "__main__":
    print("News Summary Agent\n")

    tools = [search, summarize_text, estimate_reading_time, estimate_reading_level]

    template = """
    You are an intelligent agent. Always follow this format strictly:

    Question: {input}
    Thought: your reasoning
    Action: one of [{tool_names}]
    Action Input: the input to the action
    Observation: result of the action
    ... (repeat as needed)
    Thought: I now know the final answer
    Final Answer: A markdown-formatted list with one bullet per article. Each bullet should include:

        - The article title as a hyperlink
        - A one-sentence summary (max 20 words)
        - The reading level (e.g., "Grade level: 9.5")
        

    DO NOT skip the format or speak outside it.

    IMPORTANT: Do NOT write any explanations or responses unless explicitly in the format above. 
    Never write normal sentences or summaries outside the required ReAct format. Do not summarize your steps unless it is in the Final Answer.

    Available tools:
    {tools}

    Begin!
    Question: {input}
    Thought: {agent_scratchpad}
    """

    prompt = PromptTemplate.from_template(template=template).partial(
        tools=render_text_description(tools),
        tool_names=", ".join([t.name for t in tools]),
    )

    llm = ChatOpenAI(
        temperature=0,
        stop=["\nObservation", "Observation"],
        callbacks=[AgentCallbackHandler()],
    )

    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["agent_scratchpad"]),
        }
        | prompt
        | llm
        | ReActSingleInputOutputParser()
    )

    intermediate_steps = []

    topic = input("Enter a topic you'd like to learn about: ").strip()
    user_question = f"Find the latest news about {topic}, summarize each article, and estimate how long it would take to read and what reading level is required."

    agent_step: Union[AgentAction, AgentFinish] = ""

    while not isinstance(agent_step, AgentFinish):
        agent_step = agent.invoke(
            {
                "input": user_question,
                "agent_scratchpad": intermediate_steps,
            }
        )
        print(f"\nAgent Step: {agent_step}\n")

        if isinstance(agent_step, AgentAction):
            tool_name = agent_step.tool
            tool_input = str(agent_step.tool_input)
            tool_to_use = find_tool_by_name(tools, tool_name)
        
            print(f"🔧 Using tool: {tool_name}")
            observation = invoke_tool(tool_to_use, tool_input)
        
            # Store Tavily results if this is the Tavily tool
            if tool_name == "tavily_search_results_json":
                tavily_raw = observation
        
            print(f"🔍 Observation: {observation}\n")
            intermediate_steps.append((agent_step, str(observation)))

    # -------------------------
    # FINAL ANSWER PARSE
    # -------------------------
    if isinstance(agent_step, AgentFinish):
        print("Final Answer (Markdown Format):\n")

        if tool_name == "tavily_search_results_json":
            try:
                results = json.loads(observation)
                bullet_points = []

                for i, item in enumerate(results["results"], 1):
                    title = item.get("title", "Untitled").strip()
                    content = item.get("content", "")
                    url = item.get("url", "#")

                    summary = summarize_text.func(content)
                    short_summary = " ".join(summary.split()[:20]) + ("..." if len(summary.split()) > 20 else "")
                    reading_level = estimate_reading_level.func(summary)
                    reading_time = estimate_reading_time.func(summary)

                    bullet = f"""- [{title}]({url})  
      Reading time: {reading_time}  
      Reading level: {reading_level}  
      Summary: {short_summary}"""
                    bullet_points.append(bullet)

                print("\n".join(bullet_points))

            except Exception as e:
                print("Could not parse or format final results:", e)
        else:
            print(agent_step.return_values["output"])

News Summary Agent



Enter a topic you'd like to learn about:  Raised Beds


***Prompt to LLM was:***
Human: 
    You are an intelligent agent. Always follow this format strictly:

    Question: Find the latest news about Raised Beds, summarize each article, and estimate how long it would take to read and what reading level is required.
    Thought: your reasoning
    Action: one of [tavily_search_results_json, summarize_text, estimate_reading_time, estimate_reading_level]
    Action Input: the input to the action
    Observation: result of the action
    ... (repeat as needed)
    Thought: I now know the final answer
    Final Answer: A markdown-formatted list with one bullet per article. Each bullet should include:

        - The article title as a hyperlink
        - A one-sentence summary (max 20 words)
        - The reading level (e.g., "Grade level: 9.5")


    DO NOT skip the format or speak outside it.

    IMPORTANT: Do NOT write any explanations or responses unless explicitly in the format above. 
    Never write normal sentences or summaries outside 

## Final Answer Display 

This section displays the agent’s final answer in Markdown. It loops through the retrieved articles, summarizes each one, estimates the reading level, and formats the output with:

- A clickable article title
- The reading level
- A short summary (max 20 words)

If an error occurs, it falls back to printing the raw output.

In [25]:
from IPython.display import display, Markdown

if isinstance(agent_step, AgentFinish):
    print("Final Answer:\n")

    try:
        results = tavily_raw if isinstance(tavily_raw, list) else []

        for item in results:
            title = item.get("title", "Untitled").strip()
            url = item.get("url", "#").strip()
            content = item.get("content", "")

            summary = summarize_text.func(content)
            short_summary = " ".join(summary.split()[:20]) + ("..." if len(summary.split()) > 20 else "")
            reading_level = estimate_reading_level.func(summary)

            display(Markdown(f"- [{title}]({url})  \n  - Reading Level: {reading_level}  \n  - Summary: {short_summary}"))

    except Exception as e:
        print("Failed to format final answer:", e)
        print("LLM Output:", agent_step.return_values["output"])

Final Answer:



- [Consider Raised Beds for this Year's Garden | Rural Lifestyle News](https://www.ourcoop.com/news/rural-lifestyle/consider-raised-beds-for-this-year%E2%80%99s-garden)  
  - Reading Level: Grade level: 13.9  
  - Summary: Raised beds offer many advantages. They provide better soil control, offer improved drainage, and in some cases, can extend the...

- [Big News from the Garden! We've added brand new raised beds to ...](https://www.facebook.com/ARCActivitiesRecreationandCare/videos/big-news-from-the-gardenweve-added-brand-new-raised-beds-to-our-garden-time-to-g/600641923042228/)  
  - Reading Level: Grade level: 7.2  
  - Summary: Big News from the Garden! We've added brand new raised beds to our garden. Time to get ready for a...

- [RAISED BEDS - High Country Living](https://highcountryliving.net/garden/raised-beds/)  
  - Reading Level: Grade level: 15.6  
  - Summary: Tags: [GARDEN](https://highcountryliving.net/tag/garden/), [raised beds](https://highcountryliving.net/tag/raised-beds/) ## LATEST BLOG POST ##### [Top Tips](https://highcountryliving.net/top-tips/) **GOOD NEWS RANCH** **LOCATION || GUFFEY, CO** **ELEVATION ||...